
# **Classification by BertForSequenceClassification**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pandas as pd

# Load the DataFrame
file_path = "/content/drive/MyDrive/Colab_Notebooks2/train_en.tsv"


# Use read_csv with a tab delimiter
df = pd.read_csv(file_path, delimiter='\t')


test start

In [24]:
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
import pandas as pd


In [25]:
# Preprocessing
import re

def clean_text(text):
  text = text.lower()  # Convert to lowercase
  text = re.sub(r"[^a-z0-9\s]", "", text)  # Remove non-alphanumeric characters and spaces
  text = re.sub(r"\s+", " ", text)  # Remove extra spaces
  return text

# Apply the cleaning function to each sentence
df['cleaned_sentence'] = df['sentence'].apply(clean_text)

In [26]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [27]:
#use preprocessed sentences
sentences = df['cleaned_sentence'].values

In [28]:
# encode all tokenized the sentences
input_ids = []

for sentence in sentences:
    encoded_sentence = tokenizer.encode(sentence)
    input_ids.append(encoded_sentence)

# example
print(sentences[0])
print(input_ids[0])

gone are the days when they led the world in recessionbusting
[101, 2908, 2024, 1996, 2420, 2043, 2027, 2419, 1996, 2088, 1999, 19396, 8286, 3436, 102]


In [29]:
# We will use some utility function from tensorflow
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 150

#Padding the input to the max length that is 64
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

# example
print(sentences[0])
print(input_ids[0])

gone are the days when they led the world in recessionbusting
[  101  2908  2024  1996  2420  2043  2027  2419  1996  2088  1999 19396
  8286  3436   102     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]


In [30]:
# Creating the attention masks
attention_masks = []
for i in input_ids:
    att_mask = [int(token_id > 0) for token_id in i]
    attention_masks.append(att_mask)

print(attention_masks[0])
print(attention_masks[1])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [32]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
# Convert labels to binary values of 0 and 1
label_mapping = {'OBJ': 0, 'SUBJ': 1}

# Use map function to replace values in the "label" column
df['labelBinary'] = df['label'].map(label_mapping)

labels = df['labelBinary'].values
print(labels)

[1 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1
 0 1 0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 0 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0
 1 0 0 1 0 1 0 1 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0
 0 0 1 0 0 0 1 0 0 0 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 0 0 0 1 1 1 0 0 1 0 0 1
 1 0 0 1 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 0 1 0 0 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 1
 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 1 0
 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1
 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 1 0 0 0 1
 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0
 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 0 0 1 0 0 0 1 1 0 1
 1 0 1 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 0
 1 0 0 0 1 0 0 0 0 1 1 0 

In [34]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Create a TensorDataset
dataset = TensorDataset(torch.tensor(input_ids), torch.tensor(attention_masks), torch.tensor(labels))

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [35]:
print(torch.tensor(input_ids).shape)
print(torch.tensor(attention_masks).shape)
print(torch.tensor(labels).shape)

torch.Size([830, 150])
torch.Size([830, 150])
torch.Size([830])


In [36]:
# Set up training parameters
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [37]:
# Train the model
model.train()
for epoch in range(2):  # Adjust the number of epochs as needed
    for batch in dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.3198506534099579
Epoch 2, Loss: 0.09345953911542892


# Evaluation of BertForSequenceClassification

In [38]:
# load test data
file_path2 = "/content/drive/MyDrive/Colab_Notebooks2/dev_test_en.tsv"


# Use read_csv with a tab delimiter
df_test = pd.read_csv(file_path2, delimiter='\t')

# Load the DataFrame to confirm it's loaded correctly
print(df_test.head())

                            sentence_id  \
0  8745d4da-91c9-4538-acee-b0e7b1c413fd   
1  43de04ad-d0ac-4852-9b4e-cf0bca066188   
2  e00b66ee-720a-47e3-a0fb-0e2445b89af6   
3  0b95d635-f821-45dd-9f33-b05d63629195   
4  5ba3117b-3ef9-4815-acb4-a263d3c816bc   

                                            sentence label  
0  Who will redistribute the hoarded wealth that ...  SUBJ  
1  What we don’t need is the indiscriminate influ...  SUBJ  
2  The Social Distance Between Us shows every sig...   OBJ  
3  History shows that McCarthy and McConnell, lik...   OBJ  
4  So while it’s not hard to reach a banal point ...  SUBJ  


In [39]:
# Apply the cleaning function to each sentence
df_test['cleaned_sentence'] = df_test['sentence'].apply(clean_text)
sentences_test = df_test['cleaned_sentence'].values

# Use map function to replace values in the "label" column
label_mapping = {'OBJ': 0, 'SUBJ': 1}
df_test['labelBinary'] = df_test['label'].map(label_mapping)
labels_test = df_test['labelBinary'].values


In [40]:
import torch
from transformers import XLMRobertaTokenizer
from keras.preprocessing.sequence import pad_sequences

# encode all tokenized the sentences
input_ids_test = []

for sentence in sentences_test:
    encoded_sentence = tokenizer.encode(sentence)
    input_ids_test.append(encoded_sentence)


MAX_LEN = 150
#Padding the input to the max length that is 64
input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")


# Creating the attention masks
attention_masks_test = []
for i in input_ids_test:
    att_mask = [int(token_id > 0) for token_id in i]
    attention_masks_test.append(att_mask)

In [41]:
# Create a TensorDataset
dataset_test = TensorDataset(torch.tensor(input_ids_test), torch.tensor(attention_masks_test), torch.tensor(labels_test))

# Create a DataLoader
dataloader_test = DataLoader(dataset_test, batch_size=8, shuffle=True)

In [42]:
print(torch.tensor(input_ids_test).shape)
print(torch.tensor(attention_masks_test).shape)
print(torch.tensor(labels_test).shape)

torch.Size([243, 150])
torch.Size([243, 150])
torch.Size([243])


In [43]:
# Set the model to evaluation mode
model.eval()

# Initialize variables to store predictions and true labels
predictions = []
true_labels = []

# Evaluate the model on the test set
with torch.no_grad():
    for batch in dataloader_test:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute evaluation metrics
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(true_labels, predictions)
classification_rep = classification_report(true_labels, predictions)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.7654320987654321
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.89      0.78       116
           1       0.86      0.65      0.74       127

    accuracy                           0.77       243
   macro avg       0.78      0.77      0.76       243
weighted avg       0.79      0.77      0.76       243



# **Classification by XLMRobertaForSequenceClassification**

In [11]:
import pandas as pd

# Load the DataFrame
file_path = "/content/drive/MyDrive/Colab_Notebooks2/train_en.tsv"

# Use read_csv with a tab delimiter
df = pd.read_csv(file_path, delimiter='\t')


In [12]:
# Preprocessing
import re

def clean_text(text):
  text = text.lower()  # Convert to lowercase
  text = re.sub(r"[^a-z0-9\s]", "", text)  # Remove non-alphanumeric characters and spaces
  text = re.sub(r"\s+", " ", text)  # Remove extra spaces
  return text

# Apply the cleaning function to each sentence
df['cleaned_sentence'] = df['sentence'].apply(clean_text)

In [13]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, AdamW

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base', language='en', do_lower_case=True)

In [14]:
#use preprocessed sentences
sentences = df['cleaned_sentence'].values

# encode all tokenized the sentences
input_ids = []

for sentence in sentences:
    encoded_sentence = tokenizer.encode(sentence)
    input_ids.append(encoded_sentence)

# We will use some utility function from tensorflow
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 150

#Padding the input to the max length that is 64
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

# Creating the attention masks
attention_masks = []
for i in input_ids:
    att_mask = [int(token_id > 0) for token_id in i]
    attention_masks.append(att_mask)

from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, AdamW

model = XLMRobertaForSequenceClassification.from_pretrained(
          #'xlm-roberta-large'
         'xlm-roberta-base',
         num_labels=2,
         output_attentions = False,
         output_hidden_states = False,)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Convert labels to binary values of 0 and 1
label_mapping = {'OBJ': 0, 'SUBJ': 1}

# Use map function to replace values in the "label" column
df['labelBinary'] = df['label'].map(label_mapping)

labels = df['labelBinary'].values


In [16]:

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Create a TensorDataset
dataset = TensorDataset(torch.tensor(input_ids), torch.tensor(attention_masks), torch.tensor(labels))

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Set up training parameters
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [17]:

# Train the model
model.train()
for epoch in range(2):  # Adjust the number of epochs as needed
    for batch in dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


Epoch 1, Loss: 0.5407558083534241
Epoch 2, Loss: 0.5217330455780029


# Evaluation of XLMRobertaForSequenceClassification

In [18]:

# load test data
file_path2 = "/content/drive/MyDrive/Colab_Notebooks2/dev_test_en.tsv"


# Use read_csv with a tab delimiter
df_test = pd.read_csv(file_path2, delimiter='\t')

# Load the DataFrame to confirm it's loaded correctly
print(df_test.head())


# Apply the cleaning function to each sentence
df_test['cleaned_sentence'] = df_test['sentence'].apply(clean_text)
sentences_test = df_test['cleaned_sentence'].values

# Use map function to replace values in the "label" column
label_mapping = {'OBJ': 0, 'SUBJ': 1}
df_test['labelBinary'] = df_test['label'].map(label_mapping)
labels_test = df_test['labelBinary'].values


import torch
from transformers import XLMRobertaTokenizer
from keras.preprocessing.sequence import pad_sequences

# encode all tokenized the sentences
input_ids_test = []

for sentence in sentences_test:
    encoded_sentence = tokenizer.encode(sentence)
    input_ids_test.append(encoded_sentence)


MAX_LEN = 150
#Padding the input to the max length that is 64
input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")


# Creating the attention masks
attention_masks_test = []
for i in input_ids_test:
    att_mask = [int(token_id > 0) for token_id in i]
    attention_masks_test.append(att_mask)

# Create a TensorDataset
dataset_test = TensorDataset(torch.tensor(input_ids_test), torch.tensor(attention_masks_test), torch.tensor(labels_test))

# Create a DataLoader
dataloader_test = DataLoader(dataset_test, batch_size=8, shuffle=True)


# Set the model to evaluation mode
model.eval()

# Initialize variables to store predictions and true labels
predictions = []
true_labels = []

# Evaluate the model on the test set
with torch.no_grad():
    for batch in dataloader_test:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute evaluation metrics
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(true_labels, predictions)
classification_rep = classification_report(true_labels, predictions)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)

                            sentence_id  \
0  8745d4da-91c9-4538-acee-b0e7b1c413fd   
1  43de04ad-d0ac-4852-9b4e-cf0bca066188   
2  e00b66ee-720a-47e3-a0fb-0e2445b89af6   
3  0b95d635-f821-45dd-9f33-b05d63629195   
4  5ba3117b-3ef9-4815-acb4-a263d3c816bc   

                                            sentence label  
0  Who will redistribute the hoarded wealth that ...  SUBJ  
1  What we don’t need is the indiscriminate influ...  SUBJ  
2  The Social Distance Between Us shows every sig...   OBJ  
3  History shows that McCarthy and McConnell, lik...   OBJ  
4  So while it’s not hard to reach a banal point ...  SUBJ  
Accuracy: 0.4773662551440329
Classification Report:
              precision    recall  f1-score   support

           0       0.48      1.00      0.65       116
           1       0.00      0.00      0.00       127

    accuracy                           0.48       243
   macro avg       0.24      0.50      0.32       243
weighted avg       0.23      0.48      0.31       2

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
